In [9]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

import re
import os
import pickle
from enum import Enum

In [11]:
class YouTubeAPIConstants:
    TOKEN_FILE = 'token.pickle'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [32]:
class Authenticate:
    """Handle the YouTube authentication process."""
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    __TOKEN_FILE = YouTubeAPIConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIConstants.API_VERSION
    __SCOPES = YouTubeAPIConstants.SCOPES
    
    def __init__(self, client_secrets_file: str , api_token_path: str = ''):
        """Create the auth object."""
        self.__credentials = None
        
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        
        if not api_token_path or not os.path.exists(api_token_path):
            self.__api_token_path = self.__get_default_api_token_path()
        else:
            self.__api_token_path = api_token_path
            
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
        
    def __get_default_api_token_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        api_token_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return api_token_path
    
    def __authenticate_youtube(self):
        """Authenticate the YouTube API."""
        if os.path.exists(self.__api_token_path):
            with open(self.__api_token_path, "rb") as token:
                self.__credentials = pickle.load(token)
        # if there are no (valid) credentials availablle, let the user log in.
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
#                 self.__credentials = flow.run_local_server(host='localhost', port=0)
            # save the credentials for the next run
            with open(self.__api_token_path, "wb") as token:
                pickle.dump(self.__credentials, token)

        return build(self.__API_SERVICE_NAME, self.__API_VERSION, credentials=self.__credentials)
    
    def authenticate(self):
        try:
            youtube_client =  self.__authenticate_youtube()
        except RefreshError as e:
            #delete token
            raise Exception('The token is expired. Kindly generate a new one.')
        else:
            return youtube_client

In [33]:
client_secrets_file='/home/lyle/Downloads/client_secret.json'
auth = Authenticate(client_secrets_file).authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=690018745938-f53a63e0l2kr7botgudiigblkd87vdlt.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A46105%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=4mxOfksQC6gChC23plVxTaxA4NEsVi&access_type=offline


In [35]:
from google_auth_oauthlib.flow import Flow

# Create the flow using the client secrets file from the Google API
# Console.
client_secrets_file='/home/lyle/Downloads/client_secret.json'
flow = Flow.from_client_secrets_file(
    client_secrets_file,
    scopes=["https://www.googleapis.com/auth/youtube.force-ssl"],
    redirect_uri='urn:ietf:wg:oauth:2.0:oob')

# Tell the user to go to the authorization URL.
auth_url, state = flow.authorization_url(prompt='consent')

print('Please go to this URL: {}'.format(auth_url))

# The user will get an authorization code. This code is used to get the
# access token.
code = input('Enter the authorization code: ')
flow.fetch_token(code=code)

# You can use flow.credentials, or you can just get a requests session
# using flow.authorized_session.
session = flow.authorized_session()

Please go to this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=690018745938-f53a63e0l2kr7botgudiigblkd87vdlt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=AlfjRltsyM7gJPjEY2wgdeJidWa7wy&prompt=consent&access_type=offline
Enter the authorization code: 4/1AWtgzh7R-qGGJi0jY2Mt2sowva07gCyCKFxAs7r5uYY30ct4WFYdI-SMpCs


In [36]:
credentials = flow.credentials

In [37]:
credentials

In [38]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [40]:
from googleapiclient.discovery import build 

youtube = build(
      API_SERVICE_NAME, API_VERSION, credentials=credentials)

In [41]:
youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id='hG8KEfG5lwg'
    ).execute()

{'kind': 'youtube#videoListResponse',
 'etag': 'vatIIxv7G5YKkiyNEkBNwVN2AI0',
 'items': [{'kind': 'youtube#video',
   'etag': 'UuhxuTnD4VgtYFq6LNbNFrJ7no0',
   'id': 'hG8KEfG5lwg',
   'snippet': {'publishedAt': '2023-02-16T23:00:52Z',
    'channelId': 'UCsgPO6cNV0wBG-Og3bUZoFA',
    'title': 'Gulag - The Story | Part 2: Propagation - 1934 - 1945 | Free Documentary History',
    'description': "Gulag - The Story - Part 2: Propagation - 1934 - 1945 | History Documentary\n\nWatch 'Gulag  - The Story: Part 3' here: https://youtu.be/dBjcT0QxSS0\n\nA major political, historical, human and economic fact of the 20th century, the Gulag, the extremely punitive Soviet concentration camp system, remains largely unknown.\nThe history of the Gulag is long, complex and in many ways out of the ordinary. From the Revolution of 1917 to Gorbachev, touching on the civil war, the Great Terror, World War II, the Cold War and the death of Stalin, this series describes the workings of the Gulag.\nHow and why 